In [1]:
#Note: Can add synthesis params (like use_dsp48) as 3rd series params

In [2]:
#HRR_mode: noHR only (1), noHR or SoE_HR (2), SoE_HR only (3), SI (4)
#N start: 1, 1, 2, 2
#1: no HRR and NOEPS, 2 and 3: NOEPS given, HRR calculated, 4: HRR given and corrected if needed, NOEPS calculated

In [3]:
import sys
import math
import subprocess
import shutil
import random
import datetime
import os

import import_ipynb
from REGISTERED_Generator import REGISTERED_Gen

from MatrixMatrixMultiply_SI_V1_2 import MatrixMatrixMultiply_SI
# from MatrixMatrixMultiply_SoE_V1_1 import MatrixMatrixMultiply_SoE

# from MatrixVectorMultiply_SoE_V2_5 import MatrixVectorMultiply_SoE
# from DotProduct_SoE_V4_5 import DotProduct_SoE

# from SItoSoE_V1_10 import SItoSoE

from MatrixMatrixMultiply_SI_testbench_F_v1_0 import MatrixMatrixMultiply_SI_testbench_F

from MatrixAdd_SI_V1_2 import MatrixAdd_SI

from MatrixAdd_SI_testbench_F_v1_0 import MatrixAdd_SI_testbench_F

importing Jupyter notebook from REGISTERED_Generator.ipynb
importing Jupyter notebook from MatrixMatrixMultiply_SI_V1_2.ipynb
importing Jupyter notebook from SItoSoE_V1_10.ipynb
importing Jupyter notebook from MatrixMatrixMultiply_SoE_V1_1.ipynb
importing Jupyter notebook from MatrixVectorMultiply_SoE_V2_5.ipynb
importing Jupyter notebook from DotProduct_SoE_V4_5.ipynb
importing Jupyter notebook from DotProduct_Systolic_V1_11.ipynb
importing Jupyter notebook from DotProduct_noHR_M_V3_7.ipynb
importing Jupyter notebook from RegisteredNto1Adder_V1_1.ipynb
importing Jupyter notebook from DotProduct_noHR_C_V3_7.ipynb
importing Jupyter notebook from AdderTree_ASm_V1_20.ipynb
importing Jupyter notebook from MatrixMatrixMultiply_SI_testbench_F_v1_0.ipynb
importing Jupyter notebook from MatrixAdd_SI_V1_2.ipynb
importing Jupyter notebook from VectorAdd_SI_V1_2.ipynb
importing Jupyter notebook from VectorAdd_noHR_SoE_V1_12.ipynb
importing Jupyter notebook from MatrixAdd_SI_testbench_F_v1_0.ipynb

In [4]:
# # re_import after changes
# del sys.modules['MatrixAdd_SI_testbench_F_v1_0']
# from MatrixAdd_SI_testbench_F_v1_0 import MatrixAdd_SI_testbench_F

In [5]:
# def CEIL(a,b):
#     return (a+b-1)//b

In [6]:
#read-only globals: NoOfParameters, HRRmode, Print_To_File, Files_Location
#writed globals: parameters_result
def generate_params(i):
    global NoOfAllParametersResults, NOEPS
    if i==NoOfParameters:
        NoOfAllParametersResults += 1
        ListOfAllParametersResults.append(parameters_result.copy())
        return
    
#     print(i,parameters[i])
    param = parameters[i]
    vals = parameters_values[param]
    start = vals[0]
    end = vals[1]
    if param == "N":
        for val in range(start,end+1):
            parameters_result[param] = val
            generate_params(i+1)
    elif param in ["M","Q"]:
        for val in range(start,end+1):
            parameters_result[param] = val
            generate_params(i+1)
    elif param == "PR":
        for val in range(start,min(end,parameters_result['M'])+1):
            parameters_result["PR"] = val
            generate_params(i+1)
    elif param == "PC":
        for val in range(start,min(end,parameters_result['Q'])+1):
            parameters_result["PC"] = val
            generate_params(i+1)
    elif param == "HRR":
        for val in range(start,min(end,parameters_result['N'])+1):
            NOEPS_check = math.ceil(parameters_result["N"]/val)
            if math.ceil(parameters_result["N"]/NOEPS_check) == val:
                parameters_result["HRR"] = val
                generate_params(i+1)
            #else HRR is not valid
    elif param == "NOEPS":
        if HRRmode == 2:
            endH = parameters_result["N"]
        elif HRRmode == 3:
            endH = parameters_result["N"]-1
        for val in range(start,min(end,endH)+1):
                parameters_result["NOEPS"] = val
                generate_params(i+1)
    elif param == "MAMCS":
        if HRRmode == 1:
            NOEPS = parameters_result["N"]
        elif HRRmode in [2,3]:
            NOEPS = parameters_result["NOEPS"]
        elif HRRmode == 4:
            NOEPS = math.ceil(parameters_result["N"]/parameters_result["HRR"])
        for val in range(start,min(end,NOEPS)+1):
            parameters_result["MAMCS"] = val
            generate_params(i+1)
    elif param == "ADDER_SIZE":
        if parameters_result["MAMCS"] == NOEPS:
            parameters_result["ADDER_SIZE"] = 0
            generate_params(i+1)
        else: #MAMCS < NOEPS
            for val in range(start,min(end,math.ceil(NOEPS/parameters_result["MAMCS"]))+1):
                parameters_result["ADDER_SIZE"] = val
                generate_params(i+1)
    elif param == "ENABLE_COLUMN_LATCH":
        if math.ceil(parameters_result["M"]/parameters_result["PR"]) == 1:
            parameters_result["ENABLE_COLUMN_LATCH"] = 0
            generate_params(i+1)
        else:
            for val in range(start,end+1):
                parameters_result["ENABLE_COLUMN_LATCH"] = val
                generate_params(i+1)
    elif param == "ENABLE_ROW_LATCH":
        if math.ceil(parameters_result["Q"]/parameters_result["PC"]) == 1:
            parameters_result["ENABLE_ROW_LATCH"] = 0
            generate_params(i+1)
        else:
            for val in range(start,end+1):
                parameters_result["ENABLE_ROW_LATCH"] = val
                generate_params(i+1)
    elif param in ["ENABLE_INPUT_REGISTERS","ENABLE_INPUT_LATCH","ENABLE_SIM_OUTPUT_REGISTERS"]:
        for val in range(start,end+1):
            parameters_result[param] = val
            generate_params(i+1)

In [7]:
def work_on_random_results_CPU_Core():
    global NoOfVResultsT, NoOfCPUCoreResults, parameters_result_current
    
    CPUCoreCounter = 0
    
    for index in RandomIndices:
        if CPUCoreCounter==thisCPUCore:
            NoOfCPUCoreResults += 1
            
            parameters_result_current = ListOfAllParametersResults[index]

            if print_result_flag:
                print("{}:".format(index))
                print("Parameters: ", end='')
                print(parameters_result_current)
                print("{}:".format(index), file=report_file)
                print("Parameters: ", end='', file=report_file)
                print(parameters_result_current, file=report_file)
            
            if make_base_files_flag:
                make_files()
                NoOfVResultsT += NoOfVResults
            
            if print_result_flag or make_base_files_flag:
                print()
                print(file=report_file)
                report_file.flush()
                os.fsync(report_file.fileno())

        if CPUCoreCounter==(CPUCoresCount-1):
            CPUCoreCounter=0
        else:
            CPUCoreCounter+=1

In [8]:
def make_files():
    global top, Current_Files_Location, modules, ListOfAllVerilogParametersResults, NoOfAllVerilogParametersResults
    modules = {}
    top = function(parameters_result_current, Print_To_File, Files_Location, 1, modules)
    print("Project name: "+top)
    print("Modules: ", end='')
    print(modules)
    print("Project name: "+top, file=report_file)
    print("Modules: ", end='', file=report_file)
    print(modules, file=report_file)
    Current_Files_Location = Files_Location+top+"/"
    REGISTERED_Gen(top, Print_To_File, Current_Files_Location)
    make_prj()
    make_tcl()
    if make_Synthesize_files_flag or (has_testbench and make_Simulate_files_flag):
        select_and_work_on_verilog_params()

In [9]:
def make_prj():
    with open(Current_Files_Location+top+"_REGISTERED.prj","w") as prj_file:
        print("verilog work \""+top+"_REGISTERED.v\"", file=prj_file)
        for module in modules:
            print("verilog work \""+module+".v\"", file=prj_file)

In [10]:
def make_tcl():
    
#     if Print_To_File<=0:
#         tcl_file=sys.stdout
#     else:
#         tcl_file=open(Current_Files_Location+top+".tcl", "w")

    with open(Current_Files_Location+top+".tcl", "w") as tcl_file:

        print("set myProject \""+top+"\"\n", file=tcl_file)

        print("proc set_project_props {} {", file=tcl_file)
        print("\tproject set family \""+family+"\"", file=tcl_file)
        print("\tproject set device \""+device+"\"", file=tcl_file)
        print("\tproject set package \""+package+"\"", file=tcl_file)
        print("\tproject set speed \""+speed_grade+"\"\n}\n", file=tcl_file)

        print("proc add_source_files {} {", file=tcl_file)
        print("\txfile add \""+top+"_REGISTERED.v\"", file=tcl_file)
        for module in modules:
            print("\txfile add \""+module+".v\"", file=tcl_file)
        print("\t#Set the Top Module", file=tcl_file)
        print("\tproject set top \""+top+"_REGISTERED\"\n}\n", file=tcl_file)

        print("""set proj_exts [ list ise xise gise ]
foreach ext $proj_exts {
   set proj_name "${myProject}.$ext"
   if { [ file exists $proj_name ] } { 
      file delete $proj_name
   }
}

project new $myProject
set_project_props
add_source_files
project close""", file=tcl_file)

#     if Print_To_File>0:
#         tcl_file.close()
    
    with open(Current_Files_Location+"CreateProject.bat","w") as CPB_file:
        print("xtclsh "+top+".tcl 2> TCLinfo.log", file=CPB_file)

Verilog Parameter Related Functions:

In [11]:
def generate_verilog_params_start_end(i):
    global NoOfAllVerilogParametersResults
    if i==NoOfVeilogParameters:
        ListOfAllVerilogParametersResults.append(verilog_parameters_result.copy())
        NoOfAllVerilogParametersResults += 1
        #print(verilog_parameters_result)generate_verilog_params
        return        
    param = verilog_parameters[i]
    vals = verilog_parameters_values[param]
    start = vals[0]
    end = vals[1]
    if param in ["IN_WIDTH","INPUT_REG_DEPTH","MULT_PIPE_DEPTH"]:
        for val in range(start,end+1):
            verilog_parameters_result[param] = val
            generate_verilog_params_start_end(i+1)

In [12]:
def generate_verilog_params_directly_listed(i):
    global NoOfAllVerilogParametersResults
    if i==NoOfVeilogParameters:
        ListOfAllVerilogParametersResults.append(verilog_parameters_result.copy())
        NoOfAllVerilogParametersResults += 1
        #print(verilog_parameters_result)
        return        
    param = verilog_parameters[i]
    vals = verilog_parameters_values[param]
    if param in ["IN_WIDTH","INPUT_REG_DEPTH","MULT_PIPE_DEPTH"]:
        for val in vals:
            verilog_parameters_result[param] = val
            generate_verilog_params_directly_listed(i+1)

In [13]:
def select_and_work_on_verilog_params():
    global IW_set, verilog_parameters_result_current
    IW_set = set()
    if NoOfVResults >= NoOfAllVerilogParametersResults:
        RandomVerilogIndices = range(NoOfAllVerilogParametersResults)
    else:
        RandomVerilogIndices = random.sample(range(NoOfAllVerilogParametersResults),NoOfVResults)
    for index in RandomVerilogIndices:
        verilog_parameters_result_current = ListOfAllVerilogParametersResults[index]
        make_verilog_params_files()

In [14]:
def make_verilog_params_files():
    global VP_name_append
    VP_name_append = ""
    for param in verilog_parameters_ordered:
        if param in verilog_parameters_result_current:
            VP_name_append += "_{}".format(verilog_parameters_result_current[param])
    
    if has_testbench and make_Simulate_files_flag:
        make_Simulate()
        if Simulate_flag:
            Simulate_Project()
    
    if make_Synthesize_files_flag:
        make_SYN()
        make_TMPR()
        if Synthesize_flag:
            Synthesize_Project()

Synthesize Related Functions:

In [15]:
#current layout tested for: series 6 and 7

#-p: A specific device, such as xc6vlx240t-ff1759-1, or A generic device family, such as Virtex6

def make_SYN():
    global STMPR_file_name, SynthesizeCommand
#     if Print_To_File<=0:
#         xst_file=sys.stdout
#     else:
#         STMPR_file_name = top+"_REGISTERED"+VP_name_append
#         xst_file=open(Current_Files_Location+STMPR_file_name+".xst", "w")
#     if True:

    STMPR_file_name = top+"_REGISTERED"+VP_name_append
    with open(Current_Files_Location+STMPR_file_name+".xst", "w") as xst_file:

        print("""set -tmpdir "./"
set -xsthdpdir "xst"
run""", file=xst_file)
        print("-ifn "+top+"_REGISTERED.prj", file=xst_file)
        print("-ofn "+top+"_REGISTERED"+VP_name_append, file=xst_file)
    #     print("-ofn "+top+"_REGISTERED", end='', file=xst_file)
    #     for param in verilog_parameters_ordered:
    #         if param in verilog_parameters_result_current:
    #         print("_{}".format(verilog_parameters_result_current[param]), end='', file=xst_file)
    #     print(file=xst_file)
        print("-ofmt NGC", file=xst_file)
        print("-p "+device+speed_grade+"-"+package, file=xst_file)
        print("-top "+top+"_REGISTERED", file=xst_file)
        print("""-opt_mode Speed
-opt_level 1
-power NO
-iuc NO
-keep_hierarchy No
-netlist_hierarchy As_Optimized
-rtlview Yes
-glob_opt AllClockNets
-read_cores YES
-write_timing_constraints NO
-cross_clock_analysis NO
-hierarchy_separator /
-bus_delimiter <>
-case Maintain
-slice_utilization_ratio 100
-bram_utilization_ratio 100
-dsp_utilization_ratio 100
-lc Auto
-reduce_control_sets Auto
-fsm_extract YES -fsm_encoding Auto
-safe_implementation No
-fsm_style LUT
-ram_extract Yes
-ram_style Auto
-rom_extract Yes
-shreg_extract YES
-rom_style Auto
-auto_bram_packing NO
-resource_sharing YES
-async_to_sync NO
-shreg_min_size 2
-use_dsp48 Auto
-iobuf YES
-max_fanout 100000""", file=xst_file)
        print("-bufg "+bufg, file=xst_file)
        print("""-register_duplication YES
-register_balancing No
-optimize_primitives NO
-use_clock_enable Auto
-use_sync_set Auto
-use_sync_reset Auto
-iob Auto
-equivalent_register_removal YES
-slice_utilization_ratio_maxmargin 5""", file=xst_file)
        print("-generics {", end='', file=xst_file)
        for param in verilog_parameters_result_current:
            print(param+"={} ".format(verilog_parameters_result_current[param]), end='', file=xst_file)
        print("}", file=xst_file)

#     if Print_To_File>0:
#         xst_file.close()
    
    with open(Current_Files_Location+"Synthesize"+VP_name_append+".bat","w") as SYNB_file:
        SynthesizeCommand = "xst -intstyle silent -ifn "+STMPR_file_name+".xst -ofn "+STMPR_file_name+".syr"
        print(SynthesizeCommand, file=SYNB_file)

In [16]:
def make_TMPR():
    with open(Current_Files_Location+"TMPR"+VP_name_append+".bat","w") as TMPRB_file:
        #print("ngdbuild -intstyle silent -quiet -dd _ngo -nt timestamp -i -p "+device+speed_grade+"-"+package+" "+STMPR_file_name+".ngc "+STMPR_file_name+".ngd", file=TMPRB_file)
        print("ngdbuild -dd _ngo -nt timestamp -i -p "+device+speed_grade+"-"+package+" "+STMPR_file_name+".ngc "+STMPR_file_name+".ngd", file=TMPRB_file)
        print("map -intstyle silent -p "+device+speed_grade+"-"+package+" -w -logic_opt off -ol high -t 1 -xt 0 -register_duplication off -r 4 -global_opt off -mt off -ir off -pr off -lc off -power off -o "+STMPR_file_name+"_map.ncd "+STMPR_file_name+".ngd "+STMPR_file_name+".pcf", file=TMPRB_file)
        print("par -w -intstyle silent -ol high -mt off "+STMPR_file_name+"_map.ncd "+STMPR_file_name+".ncd "+STMPR_file_name+".pcf", file=TMPRB_file)
        print("trce -intstyle silent -v 3 -s 3 -n 3 -fastpaths -xml "+STMPR_file_name+".twx "+STMPR_file_name+".ncd -o "+STMPR_file_name+".twr "+STMPR_file_name+".pcf", file=TMPRB_file)

In [17]:
def Synthesize_Project():
    resultS = subprocess.call(SynthesizeCommand, cwd=Current_Files_Location)
    print(VP_name_append+" Synthesize Result: ", end='')
    print(VP_name_append+" Synthesize Result: ", end='', file=report_file)
    if resultS==0:
        print("Successful (0)")
        print("Successful (0)", file=report_file)
    else:
        print("Unsuccessful ({})".format(resultS))
        print("Unsuccessful ({})".format(resultS), file=report_file)

TestBench Related Functions:

In [18]:
#testbench is not included in tcl files list, user can add it to project manualy

In [19]:
test_glbl_location = "C:/Xilinx/14.7/ISE_DS/ISE/verilog/src/glbl.v"

In [20]:
def make_Simulate():
    global FuseCommand, SimulateCommand
    
    IWp = verilog_parameters_result_current["IN_WIDTH"]
    
    if IWp not in IW_set:
        IW_set.add(IWp)
        testbench_function(parameters_result_current, Print_To_File, Files_Location, verilog_parameters_result_current["IN_WIDTH"])
        with open(Current_Files_Location+top+"_test_IW{}_beh.prj".format(IWp),"w") as prj_file:
            #this may generate duplicate prj file (for same IWp), but no problem
            for module in modules:
                print("verilog work \""+module+".v\"", file=prj_file)
            print("verilog work \""+top+"_test_IW{}.v\"".format(IWp), file=prj_file)
            print("verilog work \""+test_glbl_location+"\"", file=prj_file)
    
    FuseCommand = "fuse -intstyle silent -nodebug -incremental -lib unisims_ver -lib unimacro_ver -o test_isim_beh"+VP_name_append+".exe -prj "+top+"_test_IW{}_beh.prj work.".format(IWp)+top+"_test_IW{} work.glbl".format(IWp)
    for param in verilog_parameters_result_current:
        if param!="IN_WIDTH":
            FuseCommand += " -generic_top \""+param+"={}\"".format(verilog_parameters_result_current[param])

    SimulateCommand = "test_isim_beh"+VP_name_append+".exe -intstyle silent -tclbatch isim.tcl -wdb test_isim_beh"+VP_name_append+".wdb"
        
    with open(Current_Files_Location+"Simulate"+VP_name_append+".bat","w") as Simulate_file:
        print(FuseCommand, file=Simulate_file)
        print(SimulateCommand, file=Simulate_file)

    shutil.copy("./isim.tcl",Current_Files_Location+"isim.tcl")

In [21]:
def Simulate_Project():
    resultFuse = subprocess.call(FuseCommand, cwd=Current_Files_Location)
    print(VP_name_append+" Fuse Result: ", end='')
    print(VP_name_append+" Fuse Result: ", end='', file=report_file)
    if resultFuse==0:
        print("Successful (0)")
        print("Successful (0)", file=report_file)
    else:
        print("Unsuccessful ({})".format(resultFuse))
        print("Unsuccessful ({})".format(resultFuse), file=report_file)
        return

    resultSimulateRun = subprocess.call(SimulateCommand, cwd=Current_Files_Location, shell=True)
    print(VP_name_append+" Simulate Run Return Code: ", end='')
    print(VP_name_append+" Simulate Run Return Code: ", end='', file=report_file)
    if resultSimulateRun==0:
        print("Successful (0)")
        print("Successful (0)", file=report_file)
    else:
        print("Unsuccessful ({})".format(resultSimulateRun))
        print("Unsuccessful ({})".format(resultSimulateRun), file=report_file)
        return

    #open output file and check saimulation result
    with open(Current_Files_Location+VP_name_append+"_Simulation_Result.txt","r") as Simulation_Result_file:
        resultSimulate = Simulation_Result_file.readline()
    print(VP_name_append+" Simulation Result: "+resultSimulate, end='')
    print(VP_name_append+" Simulation Result: "+resultSimulate, end='', file=report_file)

Functions Start

In [22]:
functions_list = []
functions_names_list = []
parameters_list = []
HRR_mode_list = []
verilog_parameters_list = []
has_testbench_list = []
testbench_functions_list = []

In [23]:
#MatrixMatrixMultiply_SI

functions_list.append(MatrixMatrixMultiply_SI)
functions_names_list.append("MatrixMatrixMultiply_SI")

parameters_list.append(["M","N","Q","PR","PC","HRR","MAMCS","ADDER_SIZE",
                       "ENABLE_INPUT_LATCH","ENABLE_COLUMN_LATCH","ENABLE_ROW_LATCH","ENABLE_SIM_OUTPUT_REGISTERS"])

HRR_mode_list.append(4)

verilog_parameters_list.append(["IN_WIDTH", "INPUT_REG_DEPTH", "MULT_PIPE_DEPTH"])

has_testbench_list.append(True)
testbench_functions_list.append(MatrixMatrixMultiply_SI_testbench_F)

In [24]:
# #MatrixMatrixMultiply_SoE

# functions_list.append(MatrixMatrixMultiply_SoE)
# functions_names_list.append("MatrixMatrixMultiply_SoE")

# parameters_list.append(["M","N","Q","PR","PC","NOEPS","MAMCS","ADDER_SIZE",
#                        "ENABLE_INPUT_LATCH","ENABLE_COLUMN_LATCH","ENABLE_ROW_LATCH","ENABLE_SIM_OUTPUT_REGISTERS"])

# HRR_mode_list.append(3)

# verilog_parameters_list.append(["IN_WIDTH", "INPUT_REG_DEPTH", "MULT_PIPE_DEPTH"])

# has_testbench_list.append(False)
# testbench_functions_list.append(None)

In [25]:
# #M_Type2

# functions_list.append(M_Type2)
# functions_names_list.append("M_Type2")

# HRR_mode_list.append(2)

In [26]:
# #SItoSoE

# functions_list.append(SItoSoE)
# functions_names_list.append("SItoSoE")

# parameters_list.append(["N","HRR","ENABLE_INPUT_LATCH","ENABLE_SIM_OUTPUT_REGISTERS"])

# HRR_mode_list.append(4)

# verilog_parameters_list.append(["IN_WIDTH"])

# has_testbench_list.append(False)
# testbench_functions_list.append(None)

In [27]:
#MatrixAdd_SI

functions_list.append(MatrixAdd_SI)
functions_names_list.append("MatrixAdd_SI")

parameters_list.append(["M","N","PR","HRR","ENABLE_INPUT_REGISTERS","ENABLE_INPUT_LATCH"])

HRR_mode_list.append(4)

verilog_parameters_list.append(["IN_WIDTH"])

has_testbench_list.append(True)
testbench_functions_list.append(MatrixAdd_SI_testbench_F)

Functions End

In [28]:
verilog_parameters_ordered = ['IN_WIDTH', 'INPUT_REG_DEPTH', 'MULT_PIPE_DEPTH']

In [29]:
# print("choose function:")
# i=0;
# for n in functions_names_list:
#     i+=1
#     print("{}: ".format(i)+n)
# index = int(input("Enter function number([1,{}]):".format(i)))-1

In [30]:
index = 0
# index = 1

In [31]:
print("selected function: "+functions_names_list[index])

selected function: MatrixMatrixMultiply_SI


In [32]:
notation = { #will be updated after getting values
    "ENABLE_INPUT_REGISTERS": "[0(No),1(Yes)]",
    "ENABLE_INPUT_LATCH": "[0(No),1(Yes)]",
    "ENABLE_COLUMN_LATCH": "[0(No),1(Yes)]",
    "ENABLE_ROW_LATCH": "[0(No),1(Yes)]",
    "ENABLE_SIM_OUTPUT_REGISTERS": "[0(No),1(Yes)]",
    #
    "M": ">=1",
    "Q": ">=1"
}

In [33]:
function = functions_list[index]
HRRmode = HRR_mode_list[index]
if HRRmode in [1,2]:
    notation["N"] = ">=1"
elif HRRmode in [3,4]:
    notation["N"] = ">=2"

has_testbench = has_testbench_list[index]
testbench_function = testbench_functions_list[index]

In [34]:
parameters = parameters_list[index]

In [35]:
# parameters_values = {}
# notValid = False
# for parameter in parameters:
#     if parameter == "PR" and parameters_values["M"][1] == 1:
#         parameters_values["PR"] = [1,1]
#         RSmax = 1
#         print("PR: 1")
#         continue
#     if parameter == "PC" and parameters_values["Q"][1] == 1:
#         parameters_values["PC"] = [1,1]
#         CSmax = 1
#         print("PC: 1")
#         continue
#     if parameter == "NOEPS" and (
#     (HRmode == 2 and parameters_values["N"][1] == 1) or
#     (HRmode == 3 and parameters_values["N"][1] == 2) ):
#         parameters_values["NOEPS"] = [1,1]
#         print("NOEPS: 1")
#         continue
#     if parameter == "HRR" and parameters_values["N"][1] == 2:
#         parameters_values["HRR"] = [2,2]
#         print("HRR: 2")
#         continue
#     if parameter == "MAMCS" and (
#     (HRmode == 1 and parameters_values["N"][1] == 1) or
#     (HRmode == 4 and math.ceil(parameters_values["N"][1]/parameters_values["HRR"][0]) == 1) or
#     (HRmode in [2,3] and parameters_values["NOEPS"][1] == 1) ):
#         parameters_values["MAMCS"] = [1,1]
#         NOEPSmax = 1
#         print("MAMCS: 1")
#         continue
#     if parameter == "ADDER_SIZE":
#         if parameters_values["MAMCS"][0] == NOEPSmax:
#             parameters_values["ADDER_SIZE"] = [0,0]
#             print("ADDER_SIZE: 0")
#             continue
#         elif math.ceil(NOEPSmax/parameters_values["MAMCS"][0]) == 2: #else means parameters_values["MAMCS"][0] < NOEPSmax
#             parameters_values["ADDER_SIZE"] = [2,2]
#             print("ADDER_SIZE: 2")
#             continue
#     if parameter == "ENABLE_COLUMN_LATCH" and RSmax==1:
#         parameters_values["ENABLE_COLUMN_LATCH"] = [0,0]
#         print("ENABLE_COLUMN_LATCH: 0")
#         continue
#     if parameter == "ENABLE_ROW_LATCH" and CSmax==1:
#         parameters_values["ENABLE_ROW_LATCH"] = [0,0]
#         print("ENABLE_ROW_LATCH: 0")
#         continue

#     print(parameter+": "+notation[parameter])
#     startI = int(input(parameter+" start:"))
#     endI = int(input(parameter+" end:"))
#     if parameter == "N":
#         end = endI
#         if HRmode in [1,2]:
#             start = max(startI,1)
#             notation["NOEPS"] = "[1,{}]".format(end) #2
#         elif HRmode in [3,4]:
#             start = max(startI,2)
#             notation["HRR"] = "[2,{}] (not all HRR values are acceptable)".format(end) #4
#             notation["NOEPS"] = "[1,{}]".format(end-1) #3
#         if HRmode == 1:
#             NOEPSmax = end
#             notation["MAMCS"] = "[1,{}]".format(NOEPSmax)
#     elif parameter == "M":
#         start = max(startI,1)
#         end = endI
#         notation["PR"] = "[1,{}]".format(end)
#     elif parameter == "Q":
#         start = max(startI,1)
#         end = endI
#         notation["PC"] = "[1,{}]".format(end)
#     elif parameter == "PR":
#         start = max(startI,1)
#         end = min(endI,parameters_values["M"][1])
#         RSmax = math.ceil(parameters_values["M"][1]/start)
#     elif parameter == "PC":
#         start = max(startI,1)
#         end = min(endI,parameters_values["Q"][1])
#         CSmax = math.ceil(parameters_values["Q"][1]/start)
#     elif parameter in ["ENABLE_INPUT_REGISTERS","ENABLE_INPUT_LATCH","ENABLE_COLUMN_LATCH","ENABLE_ROW_LATCH","ENABLE_SIM_OUTPUT_REGISTERS"]: #Yes,No params
#         start = max(startI,0)
#         end = min(endI,1)
#     elif parameter == "HRR": #4
#         start = max(startI,2)
#         end = min(endI,parameters_values["N"][1])
#         NOEPSmax = math.ceil(parameters_values["N"][1]/start)
#         #the value itself might be invalid, but it's a correct upper limit
#         notation["MAMCS"] = "[1,{}]".format(NOEPSmax)
#     elif parameter == "NOEPS":
#         start = max(startI,1)
#         if HRmode == 2:
#             end = min(endI,parameters_values["N"][1])
#         elif HRmode == 3:
#             end = min(endI,parameters_values["N"][1]-1)
#         NOEPSmax = end
#         notation["MAMCS"] = "[1,{}]".format(NOEPSmax)
#     elif parameter == "MAMCS":
#         start = max(startI,1)
#         end = min(endI,NOEPSmax)
#         #if start < NOEPSmax:
#         notation["ADDER_SIZE"] = "[2,{}]".format(math.ceil(NOEPSmax/start)) #needed and correct for when (start < NOEPSmax)
#     elif parameter == "ADDER_SIZE":
#         start = max(startI,2)
#         end = min(endI,math.ceil(NOEPSmax/parameters_values["MAMCS"][0]))
#     else:
#         print("unknown parameter! will break.")
#         notValid = True
#         break;
#     print("calculated start: {}".format(start))
#     print("calculated end: {}".format(end))
#     if end<start:
#         print("Invalid value range! will break.")
#         notValid = True
#         break;
#     parameters_values[parameter]=[start,end]

In [36]:
parameters_values = {'M': [5, 5], 'N': [9, 9], 'Q': [5, 5], 'PR': [2, 2], 'PC': [2, 2], 'HRR': [4, 9],
'MAMCS': [1, 3], 'ADDER_SIZE': [2, 3], 'ENABLE_INPUT_LATCH': [1, 1],
'ENABLE_COLUMN_LATCH': [1, 1], 'ENABLE_ROW_LATCH': [1, 1], 'ENABLE_SIM_OUTPUT_REGISTERS': [1, 1]}

#temp changes to values
# parameters_values["NOEPS"][0] = 1 #2
# parameters_values["NOEPS"][1] = 4
# print(parameters_values)

In [37]:
# parameters_values = {'M': [5, 5], 'N': [9, 9], 'PR': [2, 2], 'HRR': [4, 9],
# 'ENABLE_INPUT_LATCH': [1, 1], 'ENABLE_INPUT_REGISTERS': [0, 1]}

In [38]:
print("parameters values: ", end='')
print(parameters_values)

parameters values: {'M': [5, 5], 'N': [9, 9], 'Q': [5, 5], 'PR': [2, 2], 'PC': [2, 2], 'HRR': [4, 9], 'MAMCS': [1, 3], 'ADDER_SIZE': [2, 3], 'ENABLE_INPUT_LATCH': [1, 1], 'ENABLE_COLUMN_LATCH': [1, 1], 'ENABLE_ROW_LATCH': [1, 1], 'ENABLE_SIM_OUTPUT_REGISTERS': [1, 1]}


In [39]:
INPUT_REG_DEPTH_max_valid = 2
MULT_PIPE_DEPTH_max_valid = 2

verilog_notation = {
    "IN_WIDTH": ">=1",
    "INPUT_REG_DEPTH": "[0,{}]".format(INPUT_REG_DEPTH_max_valid),
    "MULT_PIPE_DEPTH": "[0,{}]".format(MULT_PIPE_DEPTH_max_valid)
}

In [40]:
verilog_parameters = verilog_parameters_list[index]

In [41]:
# verilog_parameters_values = {}
# VnotValid = False
# for parameter in verilog_parameters:
#     print(parameter+": "+verilog_notation[parameter])
#     startI = int(input(parameter+" start:"))
#     endI = int(input(parameter+" end:"))
#     if parameter == "IN_WIDTH":
#         start = max(startI,1)
#         end = endI
#     elif parameter == "INPUT_REG_DEPTH":
#         start = max(startI,0)
#         end = min(endI,INPUT_REG_DEPTH_max_valid)
#     elif parameter == "MULT_PIPE_DEPTH":
#         start = max(startI,0)
#         end = min(endI,MULT_PIPE_DEPTH_max_valid)
#     else:
#         print("unknown parameter! will break.")
#         VnotValid = True
#         break;
#     print("calculated start: {}".format(start))
#     print("calculated end: {}".format(end))
#     if end<start:
#         print("Invalid value range! will break.")
#         VnotValid = True
#         break;
#     verilog_parameters_values[parameter]=[start,end]

In [42]:
verilog_parameters_values = {'IN_WIDTH': [10, 14], 'INPUT_REG_DEPTH': [1, 1], 'MULT_PIPE_DEPTH': [0, 1]}

#temp changes

In [43]:
print("verilog parameters values: ", end='')
print(verilog_parameters_values)

verilog parameters values: {'IN_WIDTH': [10, 14], 'INPUT_REG_DEPTH': [1, 1], 'MULT_PIPE_DEPTH': [0, 1]}


In [44]:
NoOfParameters = len(parameters)
#print("no of parameters: {}".format(NoOfParameters))
parameters_result = {}

In [45]:
NoOfAllParametersResults = 0
ListOfAllParametersResults = []
#modules = {}

generate_params(0)

# print(ListOfAllParametersResults)
print("no of all possible parameters results (projects): {}".format(NoOfAllParametersResults))

no of all possible parameters results (projects): 3


Random Projects Selection Start

In [46]:
# #All Parameters Results Mode:
# RandomIndices = range(NoOfAllParametersResults)

In [47]:
#Manual Parameters Results Mode:
RandomIndices = [2, 1, 0]
# RandomIndices = [2]

In [48]:
# #Random Parameters Results Mode:

# #see NoOfRandomResults

# try:
#     del sys.modules['Generator_Random_List']
# except KeyError:
#     pass
# from Generator_Random_List import RandomIndices
# print(RandomIndices)

Random Projects Selection End

In [49]:
NoOfVeilogParameters = len(verilog_parameters)
#print("no of verilog parameters: {}".format(NoOfVeilogParameters))
verilog_parameters_result = {}

In [50]:
Print_To_File = 1

In [51]:
#bufg: page 406 of xst_v6s6_v14.5

family = "Spartan6"
device = "xc6slx150"
package = "fgg900"
speed_grade = "-3"
bufg = "16"

# family = "Virtex6"
# device = "xc6vlx760"
# package = "ff1760"
# speed_grade = "-2"
# bufg = "32"

# family = "Virtex7"
# device = "xc7v2000t"
# package = "flg1925"
# speed_grade = "-2"
# bufg = "32"

In [52]:
print_result_flag = True

make_base_files_flag = True

make_Synthesize_files_flag = True
Synthesize_flag = True

make_Simulate_files_flag = True
Simulate_flag = True

verilog_params_traverse_mode = 0 #0:start_end 1:directly_listedNoOfVResults
AllVResultsForEachProject = False #False: All,   True: NoOfRandomVResultsForEachProject
NoOfRandomVResultsForEachProject = 1

CPUCoresCount = 1
thisCPUCore = 0

In [53]:
if make_base_files_flag:
    print("selected device: "+family+" "+device+speed_grade+"-"+package)

selected device: Spartan6 xc6slx150-3-fgg900


In [54]:
ListOfAllVerilogParametersResults = []
NoOfAllVerilogParametersResults = 0

if make_base_files_flag and (make_Synthesize_files_flag or (has_testbench and make_Simulate_files_flag)):
    if verilog_params_traverse_mode == 0:
        print("verilog parameters traverse mode: start_end")
        generate_verilog_params_start_end(0)
    else:
        print("verilog parameters traverse mode: directly_listed")
        generate_verilog_params_directly_listed(0)
    if AllVResultsForEachProject or NoOfRandomVResultsForEachProject >= NoOfAllVerilogParametersResults:
        NoOfVResults = NoOfAllVerilogParametersResults
    else:
        NoOfVResults = NoOfRandomVResultsForEachProject
    print("no of all possible verilog parameters modes = {}".format(NoOfAllVerilogParametersResults))
    print("no of selected verilog parameters modes for each project = {}".format(NoOfVResults))
    #print(ListOfAllVerilogParametersResults)

verilog parameters traverse mode: start_end
no of all possible verilog parameters modes = 10
no of selected verilog parameters modes for each project = 1


In [55]:
Generator_run_name_append = "8"

Files_Location = "D:/Thesis_Results_"+functions_names_list[index]+"_"+Generator_run_name_append+"/"
if not os.path.exists(Files_Location):
    os.makedirs(Files_Location)

Files_Location += "CPU_Core{}/".format(thisCPUCore)
if not os.path.exists(Files_Location):
    os.makedirs(Files_Location)

report_file=open(Files_Location+"Report.txt", "w")

print("selected function: "+functions_names_list[index], file=report_file)
print("parameters values: ", end='', file=report_file)
print(parameters_values, file=report_file)
print("verilog parameters values: ", end='', file=report_file)
print(verilog_parameters_values, file=report_file)
print("no of all possible parameters results (projects): {}".format(NoOfAllParametersResults), file=report_file)
if make_base_files_flag:
    print("selected device: "+family+" "+device+speed_grade+"-"+package, file=report_file)
if make_base_files_flag and (make_Synthesize_files_flag or (has_testbench and make_Simulate_files_flag)):
    if verilog_params_traverse_mode == 0:
        print("verilog parameters traverse mode: start_end", file=report_file)
    else:
        print("verilog parameters traverse mode: directly_listed", file=report_file)
    print("no of all possible verilog parameters modes = {}".format(NoOfAllVerilogParametersResults), file=report_file)
    print("no of selected verilog parameters modes for each project = {}".format(NoOfVResults), file=report_file)

print("output directory: "+Files_Location)
print("output directory: "+Files_Location, file=report_file)

print("selected parameters results (projects) for all cores: ", end = '')
print(RandomIndices)
print("selected parameters results (projects) for all cores: ", end = '', file=report_file)
print(RandomIndices, file=report_file)

NoOfCPUCoreResults = 0
NoOfVResultsT = 0

start_time = datetime.datetime.now()
print("start time: {}".format(start_time)+"\n")
print("start time: {}".format(start_time)+"\n", file=report_file)

report_file.flush()
os.fsync(report_file.fileno())

if print_result_flag or make_base_files_flag:
    work_on_random_results_CPU_Core()

end_time = datetime.datetime.now()
print("end time: {}".format(end_time))
print("no of CPU Core parameters results (projects): {}".format(NoOfCPUCoreResults))
print("total no of CPU Core selected verilog parameters modes: {}".format(NoOfVResultsT))
print("end time: {}".format(end_time), file=report_file)
print("no of CPU Core parameters results (projects): {}".format(NoOfCPUCoreResults), file=report_file)
print("total no of CPU Core selected verilog parameters modes: {}".format(NoOfVResultsT), file=report_file)

report_file.close()

output directory: D:/Thesis_Results_MatrixMatrixMultiply_SI_8/CPU_Core0/
selected parameters results (projects) for all cores: [2, 1, 0]
start time: 2018-01-30 19:05:06.841277

2:
Parameters: {'M': 5, 'N': 9, 'Q': 5, 'PR': 2, 'PC': 2, 'HRR': 9, 'MAMCS': 1, 'ADDER_SIZE': 0, 'ENABLE_INPUT_LATCH': 1, 'ENABLE_COLUMN_LATCH': 1, 'ENABLE_ROW_LATCH': 1, 'ENABLE_SIM_OUTPUT_REGISTERS': 1}
Project name: MatrixMatrixMultiply_5_9_5_2PR_2PC_SI_HRx9_C1_A0_IL_CL_RL_MR
Modules: {'SItoSoE_9_HRx9_IL_OR': 'SItoSoE', 'RegisteredMultiplier': 'RegisteredMultiplier', 'DotProduct_9_S1E_HRx9_C1_A0': 'DotProduct_SoE', 'MatrixVectorMultiply_5_9_2PR_S1E_HRx9_C1_A0_VL': 'MatrixVectorMultiply_SoE', 'MatrixMatrixMultiply_5_9_5_2PR_2PC_S1E_HRx9_C1_A0_CL_RL': 'MatrixMatrixMultiply_SoE', 'MatrixMatrixMultiply_5_9_5_2PR_2PC_SI_HRx9_C1_A0_IL_CL_RL_MR': 'MatrixMatrixMultiply_SI'}
_13_1_0 Fuse Result: Successful (0)
_13_1_0 Simulate Run Return Code: Successful (0)
_13_1_0 Simulation Result: Succeeded
_13_1_0 Synthesize Resu

In [56]:
# report_file.close()